In [52]:
import os
import json
import folium
from sqlalchemy import create_engine
from geoalchemy2 import Geometry
from geoalchemy2.functions import ST_BdMPolyFromText, ST_PolygonFromText
from geoalchemy2.elements import WKTElement
from llama_index import GPTSQLStructStoreIndex, SQLDatabase
from langchain import OpenAI

In [3]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
DATABASE_NAME = os.getenv("DATABASE_NAME")
DATABASE_USER = os.getenv("DATABASE_USER")
DATABASE_PASSWORD = os.getenv("DATABASE_PASSWORD")
DATABASE_PORT = os.getenv("DATABASE_PORT")
DATABASE_HOST = os.getenv("DATABASE_HOST")

In [4]:
llm_predictor = OpenAI(model_name="gpt-4")

/workspaces/forest-analyzer/.venv/lib/python3.10/site-packages/langchain/llms/openai.py:169: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/workspaces/forest-analyzer/.venv/lib/python3.10/site-packages/langchain/llms/openai.py:623: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [5]:
connection_string = f"postgresql://{DATABASE_USER}:{DATABASE_PASSWORD}@{DATABASE_HOST}:{DATABASE_PORT}/{DATABASE_NAME}"
engine = create_engine(connection_string)


In [6]:
db = SQLDatabase(engine, include_tables=["stand_4326"])

index = GPTSQLStructStoreIndex(
    [],
    sql_database=db,
    table_name="stand_4326",
)

INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 0 tokens
/workspaces/forest-analyzer/.venv/lib/python3.10/site-packages/langchain/sql_database.py:118: UserWarning: This method is deprecated - please use `get_usable_table_names`.
  warnings.warn(


In [44]:
res = index.query(" The PostGis plugin is available in the database. The forest polygons can be found from the geometry column. Get the forest polygon with largest area as geojson?")

INFO:llama_index.indices.struct_store.sql_query:> Table desc str: Schema of table stand_4326:
Table 'stand_4326' has columns: id (INTEGER), standid (BIGINT), parcelid (BIGINT), realestateid (BIGINT), standnumber (BIGINT), standnumberextension (VARCHAR(2)), maingroup (BIGINT), subgroup (BIGINT), fertilityclass (BIGINT), soiltype (BIGINT), drainagestate (BIGINT), ditchingyear (BIGINT), thinningyear (BIGINT), developmentclass (VARCHAR(2)), standquality (BIGINT), maintreespecies (BIGINT), accessibility (BIGINT), standinfo (VARCHAR), datasource (BIGINT), growthplacedatasource (BIGINT), area (DOUBLE_PRECISION), areadecrease (DOUBLE_PRECISION), creationtime (TIMESTAMP), updatetime (TIMESTAMP), geometry (geometry(GEOMETRY,-1)) and foreign keys: .

INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 475 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 0 tokens


In [61]:
geojson = json.loads(res.extra_info["result"][0][0])

In [202]:
print(res.extra_info['sql_query'])

SELECT ST_AsGeoJSON(geometry) FROM stand_4326 ORDER BY area DESC LIMIT 1;


In [199]:
g = geojson["coordinates"][0][0][0]
loc = [round(g[0],1), round(g[1],1)]
m = folium.Map(location=loc)
loc

[60.6, 23.3]

In [200]:
folium.GeoJson(geojson).add_to(m)

In [203]:
m